# Split dataset

This is for Split data to three datasets (train, val, test)

All flows are disjointly, the test distribution is same with origin datasets

Train and Test datasets are balanced, which means every class has similar number packets

In [15]:
import os
import logging
import scapy.all as scapy
import random
from collections import defaultdict
from multiprocessing import Pool, cpu_count
import json

os.chdir('/root/data')

import logging

logging.basicConfig(       
    level=logging.INFO,            
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  
    handlers=[
        logging.FileHandler('logs/split_based_flow.log', mode='w'),  
        logging.StreamHandler()          
    ],
    force=True
)

logger = logging.getLogger()

In [16]:
k = 3 # k-folder
seed = 43 # random seed
threshold = 5 # threshold for the number of packets in a flow
test_size = 1/(k + 1) # test size for train-test split
random.seed(seed)

dataset = 'ISCX-VPN-2016'
dataset_path = 'ISCX-VPN-2016/filtered/sessions'
output_path = 'ISCX-VPN-2016/filtered/flow'

if dataset == 'ISCX-VPN-2016':
    datasets_class_name = ['aim', 'email', 'facebook', 'sftp', 'gmail', 'hangout', 'icq', 'netflix', 'scp', 'ftp', 'skype', 'spotify', 'vimeo', 'torrent', 'voipbuster', 'youtube']

In [17]:
# statistics the information of the dataset
# num_of_class = len(datasets_class_name)
num_of_original_flow = 0
num_of_filtered_flow = 0
num_of_flow_per_class = defaultdict(int) # number of flows per class, which is larger than threshold
flow_file_of_class = defaultdict(list) # list of flow files per class

def process_flow(args):
    flow, folder_path, class_name = args
    count = 0
    with scapy.PcapReader(f"{folder_path}/{flow}") as packets:
        for _ in packets:
            count += 1
            if count >= threshold:
                return class_name, f"{folder_path}/{flow}"
        
for folder in os.listdir(dataset_path): # AIM_chat_1 directory
    logger.info(f"Processing folder: {folder}")
    if dataset == 'tls' or dataset == 'vpn-app':
        class_name = folder
    else:
        class_name = next((name for name in datasets_class_name if name in folder or name.upper() in folder), None)
        if class_name is None:
            logger.warning(f"No matching class for folder: {folder}")
            continue
    logger.info(f"Class name: {class_name}")

    flows = [flow for flow in os.listdir(f"{dataset_path}/{folder}")] # all flow
    num_of_original_flow += len(flows)

    with Pool(cpu_count()) as pool:
        results = pool.map(process_flow, [(flow, f"{dataset_path}/{folder}", class_name) for flow in flows])

    for result in results:
        if result is not None:
            class_name, flow = result
            num_of_filtered_flow += 1
            num_of_flow_per_class[class_name] += 1
            flow_file_of_class[class_name].append(flow)

results = {
    'num_of_original_flow': num_of_original_flow,
    'num_of_filtered_flow': num_of_filtered_flow,
    'num_of_flow_per_class': dict(num_of_flow_per_class),
    'flow_file_of_class': dict(flow_file_of_class)
}

with open(f'statistics/{dataset}_{threshold}_flow.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

logger.info('Results saved to results.json')
logger.info('Data processing completed successfully.')


2025-12-15 12:30:38,618 - root - INFO - Processing folder: netflix
2025-12-15 12:30:38,619 - root - INFO - Class name: netflix
2025-12-15 12:30:38,678 - root - INFO - Processing folder: spotify
2025-12-15 12:30:38,679 - root - INFO - Class name: spotify
2025-12-15 12:30:38,732 - root - INFO - Processing folder: skype
2025-12-15 12:30:38,733 - root - INFO - Class name: skype
2025-12-15 12:30:38,797 - root - INFO - Processing folder: ftp
2025-12-15 12:30:38,798 - root - INFO - Class name: ftp
2025-12-15 12:30:38,856 - root - INFO - Processing folder: voipbuster
2025-12-15 12:30:38,857 - root - INFO - Class name: voipbuster
2025-12-15 12:30:38,914 - root - INFO - Processing folder: sftp
2025-12-15 12:30:38,915 - root - INFO - Class name: sftp
2025-12-15 12:30:38,964 - root - INFO - Processing folder: vimeo
2025-12-15 12:30:38,965 - root - INFO - Class name: vimeo
2025-12-15 12:30:39,020 - root - INFO - Processing folder: torrent
2025-12-15 12:30:39,021 - root - INFO - Class name: torrent


In [18]:
test_num_of_flow_per_class = {key: int(test_size * value) for key, value in num_of_flow_per_class.items()}

train_val_num_of_flow_per_class = {key: value - test_num_of_flow_per_class[key] for key, value in num_of_flow_per_class.items()}

if dataset == 'ISCX-VPN-2016': # because the latest class is too small, we use the fifth smallest class
    sorted_values = sorted(train_val_num_of_flow_per_class.values())
    min_train_num_of_flow_per_class= sorted_values[5]
else:
    min_train_num_of_flow_per_class = min(train_val_num_of_flow_per_class.values())   

train_val_num_of_flow_per_class = {key: min(min_train_num_of_flow_per_class, value) for key, value in train_val_num_of_flow_per_class.items()}

print(test_num_of_flow_per_class)
print(train_val_num_of_flow_per_class)

{'netflix': 6, 'spotify': 7, 'skype': 27, 'ftp': 15, 'voipbuster': 13, 'sftp': 0, 'vimeo': 9, 'torrent': 0, 'youtube': 10, 'hangout': 33, 'icq': 1, 'scp': 54, 'aim': 2, 'email': 5, 'facebook': 8, 'gmail': 9}
{'netflix': 21, 'spotify': 21, 'skype': 21, 'ftp': 21, 'voipbuster': 21, 'sftp': 2, 'vimeo': 21, 'torrent': 1, 'youtube': 21, 'hangout': 21, 'icq': 4, 'scp': 21, 'aim': 8, 'email': 18, 'facebook': 21, 'gmail': 21}


In [19]:
# get the test and train-val set
results = {}
train_val_flow_file_of_class = defaultdict(list)
test_flow_file_of_class = defaultdict(list)

for class_name, flow_files in flow_file_of_class.items():
    test_flow_file_of_class[class_name] = random.sample(flow_files, test_num_of_flow_per_class[class_name])

for class_name, flow_files in flow_file_of_class.items():
    train_val_flow_file_of_class[class_name] = list(set(flow_files) - set(test_flow_file_of_class[class_name]))

k_folds = defaultdict(list)
for class_name, flow_files in train_val_flow_file_of_class.items():
    random.shuffle(flow_files)
    fold_size = train_val_num_of_flow_per_class[class_name] // k
    for i in range(k):
        start = i * fold_size
        end = start + fold_size
        k_folds[class_name].append(flow_files[start:end])

for i in range(k):
    results[f'k_{i}'] = {class_name: k_folds[class_name][i] for class_name in k_folds}
results['test'] = test_flow_file_of_class

with open(f'outputs/{dataset}.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

In [20]:
# create the output directory of the test dataset
os.makedirs(f"{output_path}/test", exist_ok=True)

for class_name, flow_files in test_flow_file_of_class.items():
    logger.info(f"Copying test files of class {class_name}")
    os.makedirs(f"{output_path}/test/{class_name}", exist_ok=True)
    for flow_file in flow_files:
        command_result = os.system(f"cp {flow_file} {output_path}/test/{class_name}")

2025-12-15 12:30:56,321 - root - INFO - Copying test files of class netflix
2025-12-15 12:30:56,337 - root - INFO - Copying test files of class spotify
2025-12-15 12:30:56,350 - root - INFO - Copying test files of class skype
2025-12-15 12:30:56,441 - root - INFO - Copying test files of class ftp
2025-12-15 12:30:57,905 - root - INFO - Copying test files of class voipbuster
2025-12-15 12:30:57,936 - root - INFO - Copying test files of class sftp
2025-12-15 12:30:57,937 - root - INFO - Copying test files of class vimeo
2025-12-15 12:30:58,019 - root - INFO - Copying test files of class torrent
2025-12-15 12:30:58,020 - root - INFO - Copying test files of class youtube
2025-12-15 12:30:58,039 - root - INFO - Copying test files of class hangout
2025-12-15 12:30:58,254 - root - INFO - Copying test files of class icq
2025-12-15 12:30:58,257 - root - INFO - Copying test files of class scp
2025-12-15 12:30:59,847 - root - INFO - Copying test files of class aim
2025-12-15 12:30:59,851 - root -

In [21]:
# create the output directory of the train-val dataset
for i in range(k):
    os.makedirs(f"{output_path}/train_val_split_{i}", exist_ok=True)
    os.makedirs(f"{output_path}/train_val_split_{i}/val", exist_ok=True)
    os.makedirs(f"{output_path}/train_val_split_{i}/train", exist_ok=True)

    for class_name, flow_files in k_folds.items():
        logger.info(f"Copying val files {i} of class {class_name} in split {i}")
        os.makedirs(f"{output_path}/train_val_split_{i}/val/{class_name}", exist_ok=True)

        for flow_file in flow_files[i]:
            command_result = os.system(f"cp {flow_file} {output_path}/train_val_split_{i}/val/{class_name}")


    other_index = list(range(i)) + list(range(i+1, k))
    for j in other_index:
        for class_name, flow_files in k_folds.items():
            logger.info(f"Copying train files {j} of class {class_name} in split {i}")
            os.makedirs(f"{output_path}/train_val_split_{i}/train/{class_name}", exist_ok=True)

            for flow_file in flow_files[j]:
                command_result = os.system(f"cp {flow_file} {output_path}/train_val_split_{i}/train/{class_name}")


2025-12-15 12:31:02,531 - root - INFO - Copying val files 0 of class netflix in split 0
2025-12-15 12:31:02,543 - root - INFO - Copying val files 0 of class spotify in split 0
2025-12-15 12:31:02,580 - root - INFO - Copying val files 0 of class skype in split 0
2025-12-15 12:31:02,590 - root - INFO - Copying val files 0 of class ftp in split 0
2025-12-15 12:31:02,801 - root - INFO - Copying val files 0 of class voipbuster in split 0
2025-12-15 12:31:04,222 - root - INFO - Copying val files 0 of class sftp in split 0
2025-12-15 12:31:04,223 - root - INFO - Copying val files 0 of class vimeo in split 0
2025-12-15 12:31:04,311 - root - INFO - Copying val files 0 of class torrent in split 0
2025-12-15 12:31:04,311 - root - INFO - Copying val files 0 of class youtube in split 0
2025-12-15 12:31:04,352 - root - INFO - Copying val files 0 of class hangout in split 0
2025-12-15 12:31:04,376 - root - INFO - Copying val files 0 of class icq in split 0
2025-12-15 12:31:04,378 - root - INFO - Copy